## Анализ воронок пользователей

In [ ]:
import some_funcs
from some_funcs import simple_ch_client

In [ ]:
#----------Вводить свои данные в нижние 3 переменные--------------
CH_HOST_NAME = 'rc1b-bbmw2gw58fm2dbeb.mdb.yandexcloud.net'
CH_USER      = 'z_sergey'
CH_DB_NAME   = 'metrica_data'

#-------------------------------------------
CH_PASS      = open('../.chpass.txt').read().strip()
CH_HOST      = f'https://{CH_HOST_NAME}:8443'
CH_CASERT    = 'YandexInternalRootCA.crt'

In [ ]:
my_client = simple_ch_client(CH_HOST, CH_USER, CH_PASS, CH_CASERT)

### Разминка: смотрим на пути пользователей

In [ ]:
q = f'''
SELECT
    URL,
    uniq(ClientID) as users,
    count() as hits
FROM {CH_DB_NAME}.hits
GROUP BY URL
ORDER BY users DESC
LIMIT 10
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

In [ ]:
q = f'''
SELECT
    ClientID,
    groupArray(URL) as path,
    arraySlice(groupArray(URL), 1, 5) as path_lim
FROM
    (SELECT * FROM {CH_DB_NAME}.hits ORDER BY ClientID, EventTime)
GROUP BY ClientID
LIMIT 10
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

In [ ]:
q = f'''
SELECT
    path_lim,
    count() as users
FROM
    (SELECT
        ClientID,
        arraySlice(groupArray(URL), 1, 5) as path_lim
    FROM
        (SELECT * FROM {CH_DB_NAME}.hits ORDER BY ClientID, EventTime)
    GROUP BY ClientID)
GROUP BY path_lim
ORDER BY users desc
LIMIT 10
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

Пофильтруем повторяющиеся шаги в путях, чтобы было понятнее, в каких разделах были пользователи

In [ ]:
q = f'''
SELECT
    path,
    count() as users,
    arrayEnumerate(path) as indexes,
    arrayFilter(val, index -> (index = 1) or (path[index - 1] != path[index]),
        path, indexes) as path_filt
FROM
(
    SELECT
        ClientID,
        groupArray(URL) as path
    FROM
    (
        SELECT ClientID, URL 
        FROM {CH_DB_NAME}.hits sample 0.01
        ORDER BY ClientID, EventTime
    )
    GROUP BY ClientID
)
GROUP BY path
ORDER BY users desc
LIMIT 10
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

In [ ]:
q = f'''
SELECT 
    path_filt,
    sum(users) as users
FROM
(
    SELECT
        path,
        count() as users,
        arrayEnumerate(path) as indexes,
        arrayFilter(val, index -> (index = 1) or (path[index - 1] != path[index]),
            path, indexes) as path_filt
    FROM
    (
        SELECT
            ClientID,
            groupArray(URL) as path
        FROM
        (
            SELECT ClientID, URL
            FROM {CH_DB_NAME}.hits sample 0.01
            ORDER BY ClientID, EventTime
        )
        GROUP BY ClientID
    )
    GROUP BY path
)
GROUP BY path_filt
ORDER BY users DESC
LIMIT 10
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

## Расчет funnels

#### Подход в лоб

In [ ]:
q = f'''
SELECT
    ClientID,
    max(
        URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search')) as step1_achieved,
    sequenceMatch('(?1).*(?2)')(EventTime,
        URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
        URL = 'https://supermarket.ru/basket') as step2_achieved,
    sequenceMatch('(?1).*(?2).*(?3)')(EventTime, 
        URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
        URL = 'https://supermarket.ru/basket',
        URL = 'https://supermarket.ru/order_step_1') as step3_achieved,
    sequenceMatch('(?1).*(?2).*(?3).*(?4)')(EventTime, 
        URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
        URL = 'https://supermarket.ru/basket',
        URL = 'https://supermarket.ru/order_step_1', 
        URL = 'https://supermarket.ru/order_step_2') as step4_achieved,
    sequenceMatch('(?1).*(?2).*(?3).*(?4).*(?5)')(EventTime, 
        URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
        URL = 'https://supermarket.ru/basket',
        URL = 'https://supermarket.ru/order_step_1', 
        URL = 'https://supermarket.ru/order_step_2', 
        URL = 'https://supermarket.ru/order_step_3') as step5_achieved
FROM {CH_DB_NAME}.hits
GROUP BY ClientID
LIMIT 10
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

In [ ]:
q = f'''
SELECT
    sum(step1_achieved) as step1,
    sum(step2_achieved) as step2,
    sum(step3_achieved) as step3,
    sum(step4_achieved) as step4,
    sum(step5_achieved) as step5
    --round(100.*step2/step1, 2) as step2_to_1,
    --round(100.*step3/step1, 2) as step3_to_1
FROM
    (SELECT
        ClientID,
        max(
            URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search')) as step1_achieved,
        sequenceMatch('(?1).*(?2)')(EventTime,
            URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
            URL = 'https://supermarket.ru/basket') as step2_achieved,
        sequenceMatch('(?1).*(?2).*(?3)')(EventTime, 
            URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
            URL = 'https://supermarket.ru/basket',
            URL = 'https://supermarket.ru/order_step_1') as step3_achieved,
        sequenceMatch('(?1).*(?2).*(?3).*(?4)')(EventTime, 
            URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
            URL = 'https://supermarket.ru/basket',
            URL = 'https://supermarket.ru/order_step_1', 
            URL = 'https://supermarket.ru/order_step_2') as step4_achieved,
        sequenceMatch('(?1).*(?2).*(?3).*(?4).*(?5)')(EventTime, 
            URL in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'),
            URL = 'https://supermarket.ru/basket',
            URL = 'https://supermarket.ru/order_step_1', 
            URL = 'https://supermarket.ru/order_step_2', 
            URL = 'https://supermarket.ru/order_step_3') as step5_achieved
    FROM {CH_DB_NAME}.hits
    GROUP BY ClientID)
FORMAT TabSeparatedWithNames
'''

fdf = my_client.get_clickhouse_df(q)

In [ ]:
fdf

Визуализируем

In [ ]:
from some_funcs import plot_new_funnel


In [ ]:
phases = ['Search / Catalog', 'Basket', 'Order step 1', 'Order step 2', 'Order finished']
values = fdf.iloc[0].values

In [ ]:
plot_new_funnel(phases, values)

### Пойдем другим путем

In [ ]:
q = f'''
SELECT
    ClientID,
    groupArray(cast(EventTime as UInt64)) as times,
    groupArray(URL) as urls,
    arrayFilter(time, url -> url in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'), times, urls)[1] as step1_time
FROM
    (SELECT * FROM {CH_DB_NAME}.hits ORDER BY ClientID, EventTime)
GROUP BY ClientID
LIMIT 5
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

In [ ]:
q = f'''
SELECT
    ClientID,
    groupArray(cast(EventTime as UInt64)) as times,
    groupArray(URL) as urls,
    arrayFilter(time, url -> url in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'), times, urls)[1] as step1_time,
    arrayFilter(time, url -> url = 'https://supermarket.ru/basket' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time
FROM
    (SELECT * FROM {CH_DB_NAME}.hits ORDER BY ClientID, EventTime)
GROUP BY ClientID
LIMIT 5
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

In [ ]:
q = f'''
SELECT
    countIf(step1_time != 0) as step1,
    countIf(step2_time != 0) as step2,
    countIf(step3_time != 0) as step3,
    countIf(step4_time != 0) as step4,
    countIf(step5_time != 0) as step5
FROM
    (SELECT
        ClientID,
        groupArray(cast(EventTime as UInt64)) as times,
        groupArray(URL) as urls,
        arrayFilter(time, url -> url in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'), times, urls)[1] as step1_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/basket' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_1' and step2_time != 0 and time >= step2_time, times, urls)[1] as step3_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_2' and step3_time != 0 and time >= step3_time, times, urls)[1] as step4_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_3' and step4_time != 0 and time >= step4_time, times, urls)[1] as step5_time
    FROM
        (SELECT * FROM {CH_DB_NAME}.hits ORDER BY ClientID, EventTime)
    GROUP BY ClientID)
FORMAT TabSeparatedWithNames
'''

fdf = my_client.get_clickhouse_df(q)

In [ ]:
phases = ['Search / Catalog', 'Basket', 'Order step 1', 'Order step 2', 'Order finished']
values = fdf.iloc[0].values

plot_new_funnel(phases, values)

Можем также посчитать среднее время достижения шагов

In [ ]:
q = f'''
SELECT
    medianIf(step2_time - step1_time, step2_time != 0) as step2_from_step1_mean_time,
    medianIf(step3_time - step2_time, step3_time != 0) as step3_from_step2_mean_time,
    medianIf(step4_time - step3_time, step4_time != 0) as step4_from_step3_mean_time,
    medianIf(step5_time - step4_time, step5_time != 0) as step5_from_step4_mean_time
FROM
    (SELECT
        ClientID,
        groupArray(cast(EventTime as UInt64)) as times,
        groupArray(URL) as urls,
        arrayFilter(time, url -> url in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'), times, urls)[1] as step1_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/basket' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_1' and step2_time != 0 and time >= step2_time, times, urls)[1] as step3_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_2' and step3_time != 0 and time >= step3_time, times, urls)[1] as step4_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_3' and step4_time != 0 and time >= step4_time, times, urls)[1] as step5_time
    FROM
        (SELECT * FROM {CH_DB_NAME}.hits ORDER BY ClientID, EventTime)
    GROUP BY ClientID)
FORMAT TabSeparatedWithNames
'''

my_client.get_clickhouse_df(q)

### Воронки по браузеру

In [ ]:
q = f'''
SELECT
    Browser,
    countIf(step1_time != 0) as step1,
    countIf(step2_time != 0) as step2,
    countIf(step3_time != 0) as step3,
    countIf(step4_time != 0) as step4,
    countIf(step5_time != 0) as step5
FROM
(
    SELECT
        Browser,
        ClientID,
        groupArray(cast(EventTime as UInt64)) as times,
        groupArray(URL) as urls,
        arrayFilter(time, url -> url in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'), times, urls)[1] as step1_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/basket' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_1' and step2_time != 0 and time >= step2_time, times, urls)[1] as step3_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_2' and step3_time != 0 and time >= step3_time, times, urls)[1] as step4_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_3' and step4_time != 0 and time >= step4_time, times, urls)[1] as step5_time
    FROM
        (SELECT * FROM {CH_DB_NAME}.hits ORDER BY ClientID, EventTime)
    GROUP BY Browser, ClientID
)
GROUP BY Browser
FORMAT TabSeparatedWithNames
'''

fdf = my_client.get_clickhouse_df(q)

In [ ]:
fdf.sort_values('step1', ascending=False).head()

In [ ]:
my_client.get_clickhouse_data(f'drop table if exists {CH_DB_NAME}.funnels_by_bro')

q = f'''
create table {CH_DB_NAME}.funnels_by_bro Engine = Log as
SELECT
    Browser,
    countIf(step1_time != 0) as step1,
    countIf(step2_time != 0) as step2,
    countIf(step3_time != 0) as step3,
    countIf(step4_time != 0) as step4,
    countIf(step5_time != 0) as step5
FROM
(
    SELECT
        Browser,
        ClientID,
        groupArray(cast(EventTime as UInt64)) as times,
        groupArray(URL) as urls,
        arrayFilter(time, url -> url in ('https://supermarket.ru/catalog', 'https://supermarket.ru/search'), times, urls)[1] as step1_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/basket' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_1' and step2_time != 0 and time >= step2_time, times, urls)[1] as step3_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_2' and step3_time != 0 and time >= step3_time, times, urls)[1] as step4_time,
        arrayFilter(time, url -> url = 'https://supermarket.ru/order_step_3' and step4_time != 0 and time >= step4_time, times, urls)[1] as step5_time
    FROM
        (SELECT * FROM {CH_DB_NAME}.hits ORDER BY ClientID, EventTime)
    GROUP BY Browser, ClientID
)
GROUP BY Browser
'''

my_client.get_clickhouse_data(q)